In [1]:
!pip install --upgrade google-api-python-client

Requirement already up-to-date: google-api-python-client in /home/guilherme-resende/anaconda3/lib/python3.7/site-packages (2.17.0)


In [2]:
import sys
import yaml
import glob
import os
import pandas as pd
import time
import re

from tqdm import tqdm
from googleapiclient import discovery

In [3]:
credentials = yaml.load(open("../credentials.yaml"))["perspective-api"]

/home/guilherme-resende/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  """Entry point for launching an IPython kernel.


In [4]:
def get_toxicity_score(service, text):
    try:
        if sys.getsizeof(text) > 0 and sys.getsizeof(text) < 3000:
            analyze_request = {
                'comment': {'text': text},
                'requestedAttributes': {'TOXICITY': {}}
            }
            response = service.comments().analyze(body=analyze_request).execute()
            toxicity_score = (
                response.get("attributeScores")
                .get("TOXICITY")
                .get("summaryScore")
                .get("value")
            )
        else:
            toxicity_score = None
    except:
        toxicity_score = None
    
    return toxicity_score

In [5]:
service = discovery.build(
    "commentanalyzer",
    "v1alpha1",
    developerKey=credentials["key-1"],
    discoveryServiceUrl="https://commentanalyzer.googleapis.com/$discovery/rest?version=v1alpha1",
    static_discovery=False,
)

In [6]:
input_path = "../data/02_segmented"
output_path = "../data/03_scored"
filenames = [file.split("/")[-1] for file in glob.glob(os.path.join(input_path, "*"))]

for filename in tqdm(filenames, total=len(filenames)):
    df_sentences = pd.read_csv(os.path.join(input_path, filename))
    for i, row in df_sentences.iterrows():
        df_sentences.loc[i, "score"] = get_toxicity_score(service, row.text)
        time.sleep(1)

    filename = re.findall("\d+", filename)[0]
    df_sentences.to_csv(os.path.join(output_path, f"scored-{filename}.csv"), index=False)

100%|██████████████████████████████████████████████████████████████████| 3/3 [16:39<00:00, 333.20s/it]
